In [2977]:
p_storage_dir = "../../storage"

In [2978]:
import unicodedata
import re
from pathlib import Path

import pandas as pd
from thefuzz import fuzz

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Unidades Federativas

In [2979]:
ufs = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/unidades_federativas.csv", delimiter=";")

ufs.sample(n=5)

,sigla
20,RR
22,SC
1,AL
4,CE
0,AC


### Colocar o nome completo em cada UF

In [2980]:
ufs['nome'] = ufs['sigla']

In [2981]:
uf_name_replacement_map = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MG': 'Minas Gerais',
    'MS': 'Mato Grosso do Sul',
    'MT': 'Mato Grosso',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'PR': 'Paraná',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina',
    'SE': 'Sergipe',
    'SP': 'São Paulo',
    'TO': 'Tocantins'
}

In [2982]:
ufs['nome'] = ufs['nome'].replace(uf_name_replacement_map)

In [2983]:
ufs.sample(n=10)

,sigla,nome
12,PA,Pará
5,DF,Distrito Federal
9,MG,Minas Gerais
23,SE,Sergipe
21,RS,Rio Grande do Sul
14,PE,Pernambuco
8,MA,Maranhão
18,RN,Rio Grande do Norte
0,AC,Acre
15,PI,Piauí


## Cidades

In [2984]:
cidades = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/cidades.csv", delimiter=";")

cidades.head()

,nome,sigla_uf
0,ALMEIRIM,PA
1,ANÁPOLIS,GO
2,APARECIDA DE GOIÂNIA,GO
3,Abadiânia,GO
4,Alagoinhas,BA


### Normalizar o nome das cidades 

In [2985]:
cidades['nome_normalizado'] = cidades['nome']

In [2986]:
def normalizar_string(s):
    # Transforma todos os caracteres em minúsculas
    s = s.lower()
    
    # Remove acentos das letras
    s = unicodedata.normalize("NFD", s).encode("ascii", "ignore").decode("utf-8")

    # Substitui todos os caracteres que não são letras por um espaço em branco
    s = re.sub(r'[^A-Za-z]', ' ', s)

    # Substitui espaços consecutivos por apenas um espaço
    s = re.sub(r'\s{2,}', ' ', s)

    # Remove os espaços em branco do começo e fim
    s = s.strip()

    return s

In [2987]:
cidades['nome_normalizado'] = cidades['nome_normalizado'].apply(normalizar_string)

In [2988]:
cidades[cidades['nome_normalizado'].str.contains(' ')].head(n=10)

,nome,sigla_uf,nome_normalizado
2,APARECIDA DE GOIÂNIA,GO,aparecida de goiania
6,Alto Araguaia,MT,alto araguaia
7,Alto Garças,MT,alto garcas
8,Amarante do Maranhão,MA,amarante do maranhao
14,Aparecida de Goiânia,GO,aparecida de goiania
15,Aparecida de goiânia,GO,aparecida de goiania
29,BELO HORIZONTE,MG,belo horizonte
31,BOA VISTA,RR,boa vista
33,BRASILIA-DF,DF,brasilia df
36,Barra do Corda,MA,barra do corda


### Reconciliar as cidades por UF

In [2989]:
cidades.groupby('sigla_uf')['nome'].count().sort_values(ascending=False)

sigla_uf
GO    153
MG     65
SP     55
RS     17
BA     13
DF     12
MA     11
MT     11
RJ     11
PR     10
TO     10
PA      7
MS      6
PE      6
PI      5
ES      4
CE      4
SC      3
PB      3
RN      2
SE      2
RO      1
RR      1
AL      1
AM      1
AC      1
Name: nome, dtype: int64

### Funções auxiliares

In [2990]:
def get_first_token_length(s):
    partes = s.split(" ")
    return len(partes[0])

In [2991]:
def generate_final_cidades_dataframe(nomes: list[str], nomes_ascii: list[str], sigla_uf: str | list):
    sigla = ''
    
    if isinstance(sigla_uf, str):
        sigla = [sigla_uf] * len(nomes)
    elif isinstance(sigla_uf, list):
        sigla = sigla_uf
    
    return pd.DataFrame({'nome': nomes, 'nome_ascii': nomes_ascii, 'sigla_uf': sigla})

In [2992]:
def create_aliases(row, similarity_threshold, nome_desnormalizado_map):
    nome_ascii = row["nome_ascii"]
    aliases = [nome_desnormalizado for nome_desnormalizado, nome_normalizado in
               nome_desnormalizado_map if fuzz.ratio(nome_normalizado, nome_ascii) >= similarity_threshold]
    return aliases

In [2993]:
def get_normalized_names(names: list[str]):
    return [normalizar_string(name) for name in names]

### Rio Grande do Sul - RS

In [2994]:
cidades_rs = cidades[cidades['sigla_uf'] == 'RS']

In [2995]:
cidades_rs.shape

(17, 3)

In [2996]:
cidades_rs.columns

Index(['nome', 'sigla_uf', 'nome_normalizado'], dtype='object')

In [2997]:
nome_cidades_rs = [
    'Bagé', 'Boa Vista do Buricá', 'Catuípe', 'Ijuí', 'Independência',
    'Muçum', 'Pelotas', 'Porto Alegre', 'Rio Grande', 'Santa Maria',
    'Santiago', 'Santo Ângelo', 'São Francisco de Assis',
    'Tenente Portela', 'Uruguaiana', 'Venâncio Aires'
]

nome_ascii_cidades_rs = get_normalized_names(nome_cidades_rs)

cidades_rs_final = generate_final_cidades_dataframe(nome_cidades_rs, nome_ascii_cidades_rs, 'RS')

In [2998]:
cidades_rs_final.sample(n=5)

,nome,nome_ascii,sigla_uf
3,Ijuí,ijui,RS
1,Boa Vista do Buricá,boa vista do burica,RS
9,Santa Maria,santa maria,RS
5,Muçum,mucum,RS
6,Pelotas,pelotas,RS


In [2999]:
nome_desnormalizado_map = list(zip(cidades_rs["nome"], cidades_rs["nome_normalizado"]))

cidades_rs_final['aliases'] = cidades_rs_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3000]:
cidades_rs_final

,nome,nome_ascii,sigla_uf,aliases
0,Bagé,bage,RS,[BAGÉ]
1,Boa Vista do Buricá,boa vista do burica,RS,[Boa Vista do Buricá]
2,Catuípe,catuipe,RS,[Catuípe]
3,Ijuí,ijui,RS,[Ijuí]
4,Independência,independencia,RS,[Independência]
5,Muçum,mucum,RS,[MUÇUM]
6,Pelotas,pelotas,RS,[Pelotas]
7,Porto Alegre,porto alegre,RS,[Porto Alegre]
8,Rio Grande,rio grande,RS,[Rio Grande]
9,Santa Maria,santa maria,RS,"[Santa Maria, santa maria]"


### Bahia - BA

In [3001]:
cidades_ba = cidades[cidades['sigla_uf'] == 'BA']

In [3002]:
cidades_ba

,nome,sigla_uf,nome_normalizado
4,Alagoinhas,BA,alagoinhas
46,Bom Jesus da Serra,BA,bom jesus da serra
48,Brasilia,BA,brasilia
84,Casa Nova,BA,casa nova
95,Cocos,BA,cocos
98,Correntina,BA,correntina
181,Itaberaba,BA,itaberaba
184,Itabuna,BA,itabuna
186,Itamaraju,BA,itamaraju
202,Jacobina,BA,jacobina


In [3003]:
cidades_ba = cidades_ba.drop(cidades_ba[cidades_ba['nome_normalizado'] == 'brasilia'].index)

In [3004]:
if len(cidades_ba['nome_normalizado'].unique()) == len(cidades_ba['nome_normalizado']):
    cidades_ba_final = cidades_ba[['nome', 'sigla_uf']]
    cidades_ba_final['aliases'] = cidades_ba['nome'].apply(lambda s: [s])
    cidades_ba_final['nome_ascii'] = cidades_ba['nome_normalizado']

In [3005]:
cidades_ba_final

,nome,sigla_uf,aliases,nome_ascii
4,Alagoinhas,BA,[Alagoinhas],alagoinhas
46,Bom Jesus da Serra,BA,[Bom Jesus da Serra],bom jesus da serra
84,Casa Nova,BA,[Casa Nova],casa nova
95,Cocos,BA,[Cocos],cocos
98,Correntina,BA,[Correntina],correntina
181,Itaberaba,BA,[Itaberaba],itaberaba
184,Itabuna,BA,[Itabuna],itabuna
186,Itamaraju,BA,[Itamaraju],itamaraju
202,Jacobina,BA,[Jacobina],jacobina
211,Jequié,BA,[Jequié],jequie


### Distrito Federal - DF

In [3006]:
cidades_df = cidades[cidades['sigla_uf'] == 'DF']

In [3007]:
cidades_df

,nome,sigla_uf,nome_normalizado
32,BRASILIA,DF,brasilia
33,BRASILIA-DF,DF,brasilia df
49,Braslândia,DF,braslandia
50,Brasília,DF,brasilia
56,CEILÂNDIA,DF,ceilandia
135,Gama,DF,gama
310,Recanto das Emas,DF,recanto das emas
347,Sobradinho,DF,sobradinho
348,Sobradinho - DF,DF,sobradinho df
373,Taguatinga,DF,taguatinga


In [3008]:
nome_cidades_df = [
    'Brasília', 'Brazlândia', 'Ceilândia', 'Gama', 'Recanto das Emas', 'Sobradinho', 'Taguatinga'
]

nome_ascii_cidades_df = get_normalized_names(nome_cidades_df)

cidades_df_final = generate_final_cidades_dataframe(nome_cidades_df, nome_ascii_cidades_df, 'DF')

In [3009]:
cidades_df['nome_normalizado'].apply(get_first_token_length).min()

4

In [3010]:
nome_desnormalizado_map = list(zip(cidades_df['nome'], cidades_df['nome_normalizado']))

cidades_df_final['aliases'] = cidades_df_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3011]:
cidades_df_final

,nome,nome_ascii,sigla_uf,aliases
0,Brasília,brasilia,DF,"[BRASILIA, BRASILIA-DF, Brasília, brasilia]"
1,Brazlândia,brazlandia,DF,[Braslândia]
2,Ceilândia,ceilandia,DF,[CEILÂNDIA]
3,Gama,gama,DF,[Gama]
4,Recanto das Emas,recanto das emas,DF,[Recanto das Emas]
5,Sobradinho,sobradinho,DF,"[Sobradinho, Sobradinho - DF]"
6,Taguatinga,taguatinga,DF,"[Taguatinga, Taguatinga Sul]"


### Maranhão - MA

In [3012]:
cidades_ma = cidades[cidades['sigla_uf'] == 'MA']

In [3013]:
cidades_ma

,nome,sigla_uf,nome_normalizado
8,Amarante do Maranhão,MA,amarante do maranhao
36,Barra do Corda,MA,barra do corda
82,Carolina,MA,carolina
150,Grajaú,MA,grajau
164,Icatu,MA,icatu
170,Imperatriz,MA,imperatriz
189,Itapecuru-Mirim,MA,itapecuru mirim
332,Santa Luzia,MA,santa luzia
363,São Luis,MA,sao luis
366,São Luís,MA,sao luis


In [3014]:
nome_cidades_ma = [
    'Amarante do Maranhão', 'Barra do Corda', 'Carolina', 'Grajaú', 'Icatu', 'Imperatriz', 'Itapecuru-Mirim',
    'Santa Luzia', 'São Luís', 'Viana'
]

nome_ascii_cidades_ma = get_normalized_names(nome_cidades_ma)

cidades_ma_final = generate_final_cidades_dataframe(nome_cidades_ma, nome_ascii_cidades_ma, 'MA')

In [3015]:
cidades_ma_final

,nome,nome_ascii,sigla_uf
0,Amarante do Maranhão,amarante do maranhao,MA
1,Barra do Corda,barra do corda,MA
2,Carolina,carolina,MA
3,Grajaú,grajau,MA
4,Icatu,icatu,MA
5,Imperatriz,imperatriz,MA
6,Itapecuru-Mirim,itapecuru mirim,MA
7,Santa Luzia,santa luzia,MA
8,São Luís,sao luis,MA
9,Viana,viana,MA


In [3016]:
nome_desnormalizado_map = list(zip(cidades_ma['nome'], cidades_ma['nome_normalizado']))

cidades_ma_final['aliases'] = cidades_ma_final.apply(create_aliases, args=(85, nome_desnormalizado_map), axis=1)

In [3017]:
cidades_ma_final

,nome,nome_ascii,sigla_uf,aliases
0,Amarante do Maranhão,amarante do maranhao,MA,[Amarante do Maranhão]
1,Barra do Corda,barra do corda,MA,[Barra do Corda]
2,Carolina,carolina,MA,[Carolina]
3,Grajaú,grajau,MA,[Grajaú]
4,Icatu,icatu,MA,[Icatu]
5,Imperatriz,imperatriz,MA,[Imperatriz]
6,Itapecuru-Mirim,itapecuru mirim,MA,[Itapecuru-Mirim]
7,Santa Luzia,santa luzia,MA,[Santa Luzia]
8,São Luís,sao luis,MA,"[São Luis, São Luís]"
9,Viana,viana,MA,[Viana]


### Mato Grosso - MT

In [3018]:
cidades_mt = cidades[cidades['sigla_uf'] == 'MT']

In [3019]:
cidades_mt

,nome,sigla_uf,nome_normalizado
6,Alto Araguaia,MT,alto araguaia
7,Alto Garças,MT,alto garcas
37,Barra do Garças,MT,barra do garcas
61,CUIABA,MT,cuiaba
77,Canarana,MT,canarana
102,Cuiabá,MT,cuiaba
294,Ponte Branca,MT,ponte branca
323,Rondonópolis,MT,rondonopolis
336,Santa Terezinha,MT,santa terezinha
404,cuiabá,MT,cuiaba


In [3020]:
nome_cidades_mt = [
    'Alto Araguaia', 'Alto Garças', 'Barra do Garças', 'Cuiabá', 'Canarana', 'Ponte Branca', 'Rondonópolis',
    'Santa Terezinha', 'Água Boa'
]

nome_ascii_cidades_mt = get_normalized_names(nome_cidades_mt)

cidades_mt_final = generate_final_cidades_dataframe(nome_cidades_mt, nome_ascii_cidades_mt, 'MT')

In [3021]:
nome_desnormalizado_map = list(zip(cidades_mt['nome'], cidades_mt['nome_normalizado']))

cidades_mt_final['aliases'] = cidades_mt_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3022]:
cidades_mt_final

,nome,nome_ascii,sigla_uf,aliases
0,Alto Araguaia,alto araguaia,MT,[Alto Araguaia]
1,Alto Garças,alto garcas,MT,[Alto Garças]
2,Barra do Garças,barra do garcas,MT,[Barra do Garças]
3,Cuiabá,cuiaba,MT,"[CUIABA, Cuiabá, cuiabá]"
4,Canarana,canarana,MT,[Canarana]
5,Ponte Branca,ponte branca,MT,[Ponte Branca]
6,Rondonópolis,rondonopolis,MT,[Rondonópolis]
7,Santa Terezinha,santa terezinha,MT,[Santa Terezinha]
8,Água Boa,agua boa,MT,[Água Boa]


### Rio de Janeiro - RJ

In [3023]:
cidades_rj = cidades[cidades['sigla_uf'] == 'RJ']

In [3024]:
cidades_rj

,nome,sigla_uf,nome_normalizado
47,Bom Jesus do Itabapoana,RJ,bom jesus do itabapoana
112,Duque de Caxias,RJ,duque de caxias
253,Nilopolis,RJ,nilopolis
254,Niterói,RJ,niteroi
283,Petrópolis,RJ,petropolis
307,RIO DE JANEIRO,RJ,rio de janeiro
322,Rio de Janeiro,RJ,rio de janeiro
358,São Gonçalo,RJ,sao goncalo
362,São João de Meriti,RJ,sao joao de meriti
394,Vassouras,RJ,vassouras


In [3025]:
nome_cidades_rj = [
    'Bom Jesus do Itabapoana', 'Duque de Caxias', 'Nilópolis', 'Niterói', 'Petrópolis', 'Rio de Janeiro', 'São Gonçalo',
    'São João de Meriti', 'Vassouras', 'Volta Redonda'
]

nome_ascii_cidades_rj = get_normalized_names(nome_cidades_rj)

cidades_rj_final = generate_final_cidades_dataframe(nome_cidades_rj, nome_ascii_cidades_rj, 'RJ')

In [3026]:
cidades_rj_final

,nome,nome_ascii,sigla_uf
0,Bom Jesus do Itabapoana,bom jesus do itabapoana,RJ
1,Duque de Caxias,duque de caxias,RJ
2,Nilópolis,nilopolis,RJ
3,Niterói,niteroi,RJ
4,Petrópolis,petropolis,RJ
5,Rio de Janeiro,rio de janeiro,RJ
6,São Gonçalo,sao goncalo,RJ
7,São João de Meriti,sao joao de meriti,RJ
8,Vassouras,vassouras,RJ
9,Volta Redonda,volta redonda,RJ


In [3027]:
nome_desnormalizado_map = list(zip(cidades_rj["nome"], cidades_rj["nome_normalizado"]))

cidades_rj_final['aliases'] = cidades_rj_final.apply(create_aliases, args=(90, nome_desnormalizado_map), axis=1)

In [3028]:
cidades_rj_final

,nome,nome_ascii,sigla_uf,aliases
0,Bom Jesus do Itabapoana,bom jesus do itabapoana,RJ,[Bom Jesus do Itabapoana]
1,Duque de Caxias,duque de caxias,RJ,[Duque de Caxias]
2,Nilópolis,nilopolis,RJ,[Nilopolis]
3,Niterói,niteroi,RJ,[Niterói]
4,Petrópolis,petropolis,RJ,[Petrópolis]
5,Rio de Janeiro,rio de janeiro,RJ,"[RIO DE JANEIRO, Rio de Janeiro]"
6,São Gonçalo,sao goncalo,RJ,[São Gonçalo]
7,São João de Meriti,sao joao de meriti,RJ,[São João de Meriti]
8,Vassouras,vassouras,RJ,[Vassouras]
9,Volta Redonda,volta redonda,RJ,[Volta Redonda]


### Paraná - PR

In [3029]:
cidades_pr = cidades[cidades['sigla_uf'] == 'PR']

In [3030]:
cidades_pr

,nome,sigla_uf,nome_normalizado
71,Cambé,PR,cambe
101,Cruzeiro do Sul,PR,cruzeiro do sul
104,Curitiba,PR,curitiba
152,Guarapuava,PR,guarapuava
223,Londrina,PR,londrina
237,Medianeira,PR,medianeira
255,Nova Olimpia,PR,nova olimpia
288,Pirai do Sul,PR,pirai do sul
369,São Miguel do Iguaçu,PR,sao miguel do iguacu
382,Tuneiras do Oeste,PR,tuneiras do oeste


In [3031]:
nome_cidades_pr = [
    'Cambé', 'Cruzeiro do Sul', 'Curitiba', 'Guarapuava', 'Londrina', 'Medianeira', 'Nova Olímpia', 'Piraí do Sul',
    'São Miguel do Iguaçu', 'Tuneiras do Oeste'
]

nome_ascii_cidades_pr = get_normalized_names(nome_cidades_pr)

cidades_pr_final = generate_final_cidades_dataframe(nome_cidades_pr, nome_ascii_cidades_pr, 'PR')

In [3032]:
nome_desnormalizado_map = list(zip(cidades_pr["nome"], cidades_pr["nome_normalizado"]))

cidades_pr_final['aliases'] = cidades_pr_final.apply(create_aliases, args=(90, nome_desnormalizado_map), axis=1)

In [3033]:
cidades_pr_final

,nome,nome_ascii,sigla_uf,aliases
0,Cambé,cambe,PR,[Cambé]
1,Cruzeiro do Sul,cruzeiro do sul,PR,[Cruzeiro do Sul]
2,Curitiba,curitiba,PR,[Curitiba]
3,Guarapuava,guarapuava,PR,[Guarapuava]
4,Londrina,londrina,PR,[Londrina]
5,Medianeira,medianeira,PR,[Medianeira]
6,Nova Olímpia,nova olimpia,PR,[Nova Olimpia]
7,Piraí do Sul,pirai do sul,PR,[Pirai do Sul]
8,São Miguel do Iguaçu,sao miguel do iguacu,PR,[São Miguel do Iguaçu]
9,Tuneiras do Oeste,tuneiras do oeste,PR,[Tuneiras do Oeste]


### Tocantins - TO

In [3034]:
cidades_to = cidades[cidades['sigla_uf'] == 'TO']

In [3035]:
cidades_to

,nome,sigla_uf,nome_normalizado
19,Araguaína,TO,araguaina
26,Arraias,TO,arraias
60,CRISTALÂNDIA,TO,cristalandia
99,Cristalândia,TO,cristalandia
157,Gurupi,TO,gurupi
259,PALMEIRÓPOLIS,TO,palmeiropolis
263,Palmas,TO,palmas
272,Paraíso do Tocantins,TO,paraiso do tocantins
278,Pedro Afonso,TO,pedro afonso
292,Pium,TO,pium


In [3036]:
nome_cidades_to = [
    'Araguaína', 'Arraias', 'Cristalândia', 'Gurupi', 'Palmeirópolis', 'Palmas', 'Paraíso do Tocantins',
    'Pedro Afonso', 'Pium'
]

nome_ascii_cidades_to = get_normalized_names(nome_cidades_to)

cidades_to_final = generate_final_cidades_dataframe(nome_cidades_to, nome_ascii_cidades_to, 'TO')

In [3037]:
nome_desnormalizado_map = list(zip(cidades_to["nome"], cidades_to["nome_normalizado"]))

cidades_to_final['aliases'] = cidades_to_final.apply(create_aliases, args=(95, nome_desnormalizado_map), axis=1)

In [3038]:
cidades_to_final

,nome,nome_ascii,sigla_uf,aliases
0,Araguaína,araguaina,TO,[Araguaína]
1,Arraias,arraias,TO,[Arraias]
2,Cristalândia,cristalandia,TO,"[CRISTALÂNDIA, Cristalândia]"
3,Gurupi,gurupi,TO,[Gurupi]
4,Palmeirópolis,palmeiropolis,TO,[PALMEIRÓPOLIS]
5,Palmas,palmas,TO,[Palmas]
6,Paraíso do Tocantins,paraiso do tocantins,TO,[Paraíso do Tocantins]
7,Pedro Afonso,pedro afonso,TO,[Pedro Afonso]
8,Pium,pium,TO,[Pium]


### Pará - PA

In [3039]:
cidades_pa = cidades[cidades['sigla_uf'] == 'PA']

In [3040]:
cidades_pa

,nome,sigla_uf,nome_normalizado
0,ALMEIRIM,PA,almeirim
30,BELÉM,PA,belem
44,Belém,PA,belem
203,Jacundá,PA,jacunda
313,Rendenção,PA,rendencao
400,XINGUARA,PA,xinguara
413,mARABÁ,PA,maraba


In [3041]:
nome_cidades_pa = [
    'Almeirim', 'Belém', 'Jacundá', 'Redenção', 'Xinguara', 'Marabá'
]

nome_ascii_cidades_pa = get_normalized_names(nome_cidades_pa)

cidades_pa_final = generate_final_cidades_dataframe(nome_cidades_pa, nome_ascii_cidades_pa, 'PA')

In [3042]:
nome_desnormalizado_map = list(zip(cidades_pa["nome"], cidades_pa["nome_normalizado"]))

cidades_pa_final['aliases'] = cidades_pa_final.apply(create_aliases, args=(90, nome_desnormalizado_map), axis=1)

In [3043]:
cidades_pa_final

,nome,nome_ascii,sigla_uf,aliases
0,Almeirim,almeirim,PA,[ALMEIRIM]
1,Belém,belem,PA,"[BELÉM, Belém]"
2,Jacundá,jacunda,PA,[Jacundá]
3,Redenção,redencao,PA,[Rendenção]
4,Xinguara,xinguara,PA,[XINGUARA]
5,Marabá,maraba,PA,[mARABÁ]


### Mato Grosso do Sul - MS

In [3044]:
cidades_ms = cidades[cidades['sigla_uf'] == 'MS']

In [3045]:
cidades_ms

,nome,sigla_uf,nome_normalizado
25,Arinos,MS,arinos
74,Campo Grande,MS,campo grande
110,Dourados,MS,dourados
175,Inocência,MS,inocencia
270,Paranaiba,MS,paranaiba
380,Três Lagôas,MS,tres lagoas


In [3046]:
cidades_ms = cidades_ms.drop(cidades_ms[cidades_ms['nome_normalizado'] == 'arinos'].index)

In [3047]:
nome_cidades_ms = [
    'Campo Grande', 'Dourados', 'Inocência', 'Paranaíba', 'Três Lagoas'
]

nome_ascii_cidades_ms = get_normalized_names(nome_cidades_ms)

cidades_ms_final = generate_final_cidades_dataframe(nome_cidades_ms, nome_ascii_cidades_ms, 'MS')

In [3048]:
cidades_ms_final

,nome,nome_ascii,sigla_uf
0,Campo Grande,campo grande,MS
1,Dourados,dourados,MS
2,Inocência,inocencia,MS
3,Paranaíba,paranaiba,MS
4,Três Lagoas,tres lagoas,MS


In [3049]:
nome_desnormalizado_map = list(zip(cidades_ms["nome"], cidades_ms["nome_normalizado"]))

cidades_ms_final['aliases'] = cidades_ms_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3050]:
cidades_ms_final

,nome,nome_ascii,sigla_uf,aliases
0,Campo Grande,campo grande,MS,[Campo Grande]
1,Dourados,dourados,MS,[Dourados]
2,Inocência,inocencia,MS,[Inocência]
3,Paranaíba,paranaiba,MS,[Paranaiba]
4,Três Lagoas,tres lagoas,MS,[Três Lagôas]


### Pernambuco - PE

In [3051]:
cidades_pe = cidades[cidades['sigla_uf'] == 'PE']

In [3052]:
cidades_pe

,nome,sigla_uf,nome_normalizado
20,Araripina,PE,araripina
24,Arcoverde,PE,arcoverde
83,Caruaru - PE,PE,caruaru pe
136,Garanhuns,PE,garanhuns
305,RECIFE,PE,recife
311,Recife,PE,recife


In [3053]:
nome_cidades_pe = [
    'Araripina', 'Arcoverde', 'Caruaru', 'Garanhuns', 'Recife'
]

nome_ascii_cidades_pe = get_normalized_names(nome_cidades_pe)

cidades_pe_final = generate_final_cidades_dataframe(nome_cidades_pe, nome_ascii_cidades_pe, 'PE')

In [3054]:
cidades_pe_final

,nome,nome_ascii,sigla_uf
0,Araripina,araripina,PE
1,Arcoverde,arcoverde,PE
2,Caruaru,caruaru,PE
3,Garanhuns,garanhuns,PE
4,Recife,recife,PE


In [3055]:
nome_desnormalizado_map = list(zip(cidades_pe["nome"], cidades_pe["nome_normalizado"]))

cidades_pe_final['aliases'] = cidades_pe_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3056]:
cidades_pe_final

,nome,nome_ascii,sigla_uf,aliases
0,Araripina,araripina,PE,[Araripina]
1,Arcoverde,arcoverde,PE,[Arcoverde]
2,Caruaru,caruaru,PE,[Caruaru - PE]
3,Garanhuns,garanhuns,PE,[Garanhuns]
4,Recife,recife,PE,"[RECIFE, Recife]"


### Piauí - PI

In [3057]:
cidades_pi = cidades[cidades['sigla_uf'] == 'PI']

In [3058]:
cidades_pi

,nome,sigla_uf,nome_normalizado
116,FLORIANO,PI,floriano
122,Floriano,PI,floriano
350,SÃO RAIMUNDO NONATO,PI,sao raimundo nonato
377,Teresina,PI,teresina
415,teresina,PI,teresina


In [3059]:
nome_cidades_pi = [
    'Floriano', 'São Raimundo Nonato', 'Teresina'
]

nome_ascii_cidades_pi = get_normalized_names(nome_cidades_pi)

cidades_pi_final = generate_final_cidades_dataframe(nome_cidades_pi, nome_ascii_cidades_pi, 'PI')

In [3060]:
cidades_pi_final

,nome,nome_ascii,sigla_uf
0,Floriano,floriano,PI
1,São Raimundo Nonato,sao raimundo nonato,PI
2,Teresina,teresina,PI


In [3061]:
nome_desnormalizado_map = list(zip(cidades_pi["nome"], cidades_pi["nome_normalizado"]))

cidades_pi_final['aliases'] = cidades_pi_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3062]:
cidades_pi_final

,nome,nome_ascii,sigla_uf,aliases
0,Floriano,floriano,PI,"[FLORIANO, Floriano]"
1,São Raimundo Nonato,sao raimundo nonato,PI,[SÃO RAIMUNDO NONATO]
2,Teresina,teresina,PI,"[Teresina, teresina]"


### Espírito Santo - ES

In [3063]:
cidades_es = cidades[cidades['sigla_uf'] == 'ES']

In [3064]:
cidades_es

,nome,sigla_uf,nome_normalizado
96,Colatina,ES,colatina
222,Linhares,ES,linhares
285,Pinheiros,ES,pinheiros
398,Vitória,ES,vitoria


In [3065]:
nome_cidades_es = [
    'Colatina', 'Linhares', 'Pinheiros', 'Vitória'
]

nome_ascii_cidades_es = get_normalized_names(nome_cidades_es)

cidades_es_final = generate_final_cidades_dataframe(nome_cidades_es, nome_ascii_cidades_es, 'ES')

In [3066]:
cidades_es_final

,nome,nome_ascii,sigla_uf
0,Colatina,colatina,ES
1,Linhares,linhares,ES
2,Pinheiros,pinheiros,ES
3,Vitória,vitoria,ES


In [3067]:
nome_desnormalizado_map = list(zip(cidades_es["nome"], cidades_es["nome_normalizado"]))

cidades_es_final['aliases'] = cidades_es_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3068]:
cidades_pi_final

/home/marcel/.local/share/virtualenvs/tcc-ifg-kg-HQgq8_kv/lib/python3.10/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,nome,nome_ascii,sigla_uf,aliases
0,Floriano,floriano,PI,"[FLORIANO, Floriano]"
1,São Raimundo Nonato,sao raimundo nonato,PI,[SÃO RAIMUNDO NONATO]
2,Teresina,teresina,PI,"[Teresina, teresina]"


### Ceará - CE

In [3069]:
cidades_ce = cidades[cidades['sigla_uf'] == 'CE']

In [3070]:
cidades_ce

,nome,sigla_uf,nome_normalizado
17,Aracati,CE,aracati
127,Fortaleza,CE,fortaleza
166,Iguatu,CE,iguatu
167,Iguatú,CE,iguatu


In [3071]:
nome_cidades_ce = [
    'Aracati', 'Fortaleza', 'Iguatu'
]

nome_ascii_cidades_ce = get_normalized_names(nome_cidades_ce)

cidades_ce_final = generate_final_cidades_dataframe(nome_cidades_ce, nome_ascii_cidades_ce, 'CE')

In [3072]:
cidades_ce_final

,nome,nome_ascii,sigla_uf
0,Aracati,aracati,CE
1,Fortaleza,fortaleza,CE
2,Iguatu,iguatu,CE


In [3073]:
nome_desnormalizado_map = list(zip(cidades_ce["nome"], cidades_ce["nome_normalizado"]))

cidades_ce_final['aliases'] = cidades_ce_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3074]:
cidades_ce_final

,nome,nome_ascii,sigla_uf,aliases
0,Aracati,aracati,CE,[Aracati]
1,Fortaleza,fortaleza,CE,[Fortaleza]
2,Iguatu,iguatu,CE,"[Iguatu, Iguatú]"


### Santa Catarina - SC

In [3075]:
cidades_sc = cidades[cidades['sigla_uf'] == 'SC']

In [3076]:
cidades_sc

,nome,sigla_uf,nome_normalizado
78,Canoinhas,SC,canoinhas
123,Florianópolis,SC,florianopolis
326,Salete,SC,salete


In [3077]:
nome_cidades_sc = [
    'Canoinhas', 'Florianópolis', 'Salete'
]

nome_ascii_cidades_sc = get_normalized_names(nome_cidades_sc)

cidades_sc_final = generate_final_cidades_dataframe(nome_cidades_sc, nome_ascii_cidades_sc, 'SC')

In [3078]:
cidades_sc_final

,nome,nome_ascii,sigla_uf
0,Canoinhas,canoinhas,SC
1,Florianópolis,florianopolis,SC
2,Salete,salete,SC


In [3079]:
nome_desnormalizado_map = list(zip(cidades_sc["nome"], cidades_sc["nome_normalizado"]))

cidades_sc_final['aliases'] = cidades_sc_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3080]:
cidades_sc_final

,nome,nome_ascii,sigla_uf,aliases
0,Canoinhas,canoinhas,SC,[Canoinhas]
1,Florianópolis,florianopolis,SC,[Florianópolis]
2,Salete,salete,SC,[Salete]


### Paraíba - PB

In [3081]:
cidades_pb = cidades[cidades['sigla_uf'] == 'PB']

In [3082]:
cidades_pb

,nome,sigla_uf,nome_normalizado
214,João Pessoa,PB,joao pessoa
275,Patos,PB,patos
303,Princesa Isabel,PB,princesa isabel


In [3083]:
nome_cidades_pb = [
    'João Pessoa', 'Patos', 'Princesa Isabel'
]

nome_ascii_cidades_pb = get_normalized_names(nome_cidades_pb)

cidades_pb_final = generate_final_cidades_dataframe(nome_cidades_pb, nome_ascii_cidades_pb, 'PB')

In [3084]:
cidades_pb_final

,nome,nome_ascii,sigla_uf
0,João Pessoa,joao pessoa,PB
1,Patos,patos,PB
2,Princesa Isabel,princesa isabel,PB


In [3085]:
nome_desnormalizado_map = list(zip(cidades_pb["nome"], cidades_pb["nome_normalizado"]))

cidades_pb_final['aliases'] = cidades_pb_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3086]:
cidades_pb_final

,nome,nome_ascii,sigla_uf,aliases
0,João Pessoa,joao pessoa,PB,[João Pessoa]
1,Patos,patos,PB,[Patos]
2,Princesa Isabel,princesa isabel,PB,[Princesa Isabel]


### Rio Grande do Norte - RN

In [3087]:
cidades_rn = cidades[cidades['sigla_uf'] == 'RN']

In [3088]:
cidades_rn

,nome,sigla_uf,nome_normalizado
247,NATAL,RN,natal
249,Natal,RN,natal


In [3089]:
nome_cidades_rn = ['Natal']

nome_ascii_cidades_rn = get_normalized_names(nome_cidades_rn)

cidades_rn_final = generate_final_cidades_dataframe(nome_cidades_rn, nome_ascii_cidades_rn, 'RN')

In [3090]:
cidades_rn_final

,nome,nome_ascii,sigla_uf
0,Natal,natal,RN


In [3091]:
nome_desnormalizado_map = list(zip(cidades_rn["nome"], cidades_rn["nome_normalizado"]))

cidades_rn_final['aliases'] = cidades_rn_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3092]:
cidades_rn_final

,nome,nome_ascii,sigla_uf,aliases
0,Natal,natal,RN,"[NATAL, Natal]"


### Sergipe - SE

In [3093]:
cidades_se = cidades[cidades['sigla_uf'] == 'SE']

In [3094]:
cidades_se

,nome,sigla_uf,nome_normalizado
16,Aracaju,SE,aracaju
220,Lagarto,SE,lagarto


In [3095]:
nome_cidades_se = ['Aracaju', 'Lagarto']

nome_ascii_cidades_se = get_normalized_names(nome_cidades_se)

cidades_se_final = generate_final_cidades_dataframe(nome_cidades_se, nome_ascii_cidades_se, 'SE')

In [3096]:
cidades_se_final

,nome,nome_ascii,sigla_uf
0,Aracaju,aracaju,SE
1,Lagarto,lagarto,SE


In [3097]:
nome_desnormalizado_map = list(zip(cidades_se["nome"], cidades_se["nome_normalizado"]))

cidades_se_final['aliases'] = cidades_se_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3098]:
cidades_se_final

,nome,nome_ascii,sigla_uf,aliases
0,Aracaju,aracaju,SE,[Aracaju]
1,Lagarto,lagarto,SE,[Lagarto]


### UFs que possuem apenas 1 município registrado

In [3099]:
cidades_unicas = cidades[cidades['sigla_uf'].isin(['RO', 'RR', 'AL', 'AM', 'AC'])]

In [3100]:
cidades_unicas

,nome,sigla_uf,nome_normalizado
31,BOA VISTA,RR,boa vista
229,MANAUS,AM,manaus
274,Passo de Camaragibe,AL,passo de camaragibe
299,Porto Velho,RO,porto velho
318,Rio Branco,AC,rio branco


In [3101]:
nome_cidades_unicas = ['Boa Vista', 'Manaus', 'Passo de Camaragibe', 'Porto Velho', 'Rio Branco']

nome_ascii_cidades_unicas = get_normalized_names(nome_cidades_unicas)

cidades_unicas_final = generate_final_cidades_dataframe(nome_cidades_unicas, nome_ascii_cidades_unicas, list(cidades_unicas['sigla_uf']))

In [3102]:
cidades_unicas_final

,nome,nome_ascii,sigla_uf
0,Boa Vista,boa vista,RR
1,Manaus,manaus,AM
2,Passo de Camaragibe,passo de camaragibe,AL
3,Porto Velho,porto velho,RO
4,Rio Branco,rio branco,AC


In [3103]:
nome_desnormalizado_map = list(zip(cidades_unicas["nome"], cidades_unicas["nome_normalizado"]))

cidades_unicas_final['aliases'] = cidades_unicas_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3104]:
cidades_unicas_final

,nome,nome_ascii,sigla_uf,aliases
0,Boa Vista,boa vista,RR,[BOA VISTA]
1,Manaus,manaus,AM,[MANAUS]
2,Passo de Camaragibe,passo de camaragibe,AL,[Passo de Camaragibe]
3,Porto Velho,porto velho,RO,[Porto Velho]
4,Rio Branco,rio branco,AC,[Rio Branco]


### São Paulo - SP

In [3105]:
cidades_sp = cidades[cidades['sigla_uf'] == 'SP']

In [3106]:
cidades_sp['nome_normalizado'].unique()

array(['aracatuba', 'assis', 'barretos', 'batatais', 'bauru', 'cravinhos',
       'cachoeira paulista', 'campinas', 'cacapava', 'chavantes',
       'candido mota', 'dracena', 'fartura', 'ferraz de vasconcelos',
       'franca', 'general salgado', 'guararapes', 'guarulhos', 'guara',
       'ibiuna', 'ilha solteira', 'ipua', 'itatiba', 'ituverava',
       'itapolis', 'jaboticabal', 'jacarei', 'jundiai', 'lucelia',
       'marilia', 'mococa', 'mogi das cruzes', 'parapua',
       'pereira barreto', 'piracicaba', 'presidente prudente',
       'ribeirao preto', 'registro', 'rifaina',
       'santa cruz das palmeiras', 'santo andre', 'santos',
       'sao jose dos campos', 'sao paulo', 'sertaozinho',
       'sao caetano do sul', 'sao bernardo do campo', 'sao carlos',
       'sao jose do rio preto', 'jales'], dtype=object)

In [3107]:
len(cidades_sp['nome_normalizado'].unique())

50

In [3108]:
nome_cidades_sp = [
    'Araçatuba', 'Assis', 'Barretos', 'Batatais', 'Bauru', 'Cravinhos',
    'Cachoeira Paulista', 'Campinas', 'Caçapava', 'Chavantes',
    'Cândido Mota', 'Dracena', 'Fartura', 'Ferraz de Vasconcelos',
    'Franca', 'General Salgado', 'Guararapes', 'Guarulhos', 'Guará',
    'Ibiúna', 'Ilha Solteira', 'Ipuã', 'Itatiba', 'Ituverava',
    'Itápolis', 'Jaboticabal', 'Jacareí', 'Jundiaí', 'Lucélia',
    'Marília', 'Mococa', 'Mogi das Cruzes', 'Parapuã',
    'Pereira Barreto', 'Piracicaba', 'Presidente Prudente',
    'Ribeirão Preto', 'Registro', 'Rifaina',
    'Santa Cruz das Palmeiras', 'Santo André', 'Santos',
    'São José dos Campos', 'São Paulo', 'Sertãozinho',
    'São Caetano do Sul', 'São Bernardo do Campo', 'São Carlos',
    'São José do Rio Preto', 'Jales'
]

nome_ascii_cidades_sp = get_normalized_names(nome_cidades_sp)

cidades_sp_final = generate_final_cidades_dataframe(nome_cidades_sp, nome_ascii_cidades_sp, 'SP')

In [3109]:
cidades_sp_final.sample(n=10)

,nome,nome_ascii,sigla_uf
10,Cândido Mota,candido mota,SP
20,Ilha Solteira,ilha solteira,SP
29,Marília,marilia,SP
4,Bauru,bauru,SP
19,Ibiúna,ibiuna,SP
27,Jundiaí,jundiai,SP
44,Sertãozinho,sertaozinho,SP
14,Franca,franca,SP
11,Dracena,dracena,SP
23,Ituverava,ituverava,SP


In [3110]:
nome_desnormalizado_map = list(zip(cidades_sp["nome"], cidades_sp["nome_normalizado"]))

cidades_sp_final['aliases'] = cidades_sp_final.apply(create_aliases, args=(80, nome_desnormalizado_map), axis=1)

In [3111]:
cidades_sp_final.sample(n=10)

,nome,nome_ascii,sigla_uf,aliases
24,Itápolis,itapolis,SP,[Itápolis]
12,Fartura,fartura,SP,[Fartura]
8,Caçapava,cacapava,SP,[Caçapava]
17,Guarulhos,guarulhos,SP,[Guarulhos]
27,Jundiaí,jundiai,SP,[Jundiaí]
34,Piracicaba,piracicaba,SP,[Piracicaba]
19,Ibiúna,ibiuna,SP,[Ibiuna]
35,Presidente Prudente,presidente prudente,SP,[Presidente Prudente]
22,Itatiba,itatiba,SP,[Itatiba]
32,Parapuã,parapua,SP,[Parapuã]


### Minas Gerais - MG

In [3112]:
cidades_mg = cidades[cidades['sigla_uf'] == 'MG']

In [3113]:
cidades_mg['nome_normalizado'].unique()

array(['andrelandia', 'araguari', 'araxa', 'aracuai', 'belo horizonte',
       'bambui', 'barbacena', 'canapolis', 'carmo do paranaiba',
       'cabeceira grande', 'cajuri', 'cambuquira', 'campo belo',
       'campos altos', 'caratinga', 'conselheiro lafaiete', 'divinopolis',
       'dores do indaia', 'estrela do sul', 'formiga',
       'governador valadares', 'guarda mor', 'ituiutaba', 'ipatinga',
       'itabira', 'itambacuri', 'jacuntinga', 'januaria', 'joao pinheiro',
       'lavras', 'luz', 'monte carmelo', 'montes claros', 'mantena',
       'mato verde', 'monte alegre de minas', 'nova resende',
       'patos de minas', 'paineiras', 'paracatu', 'patrocinio',
       'pocos de caldas', 'raul soares', 'riachinho', 'rio casca',
       'sacramento', 'santa fe de minas', 'santa juliana', 'sete lagoas',
       'sao goncalo do sapucai', 'sao joao del rei', 'taiobeiras',
       'timoteo', 'tres pontas', 'tupaciguara', 'uberaba', 'uberlandia',
       'unai', 'vazante'], dtype=object)

In [3114]:
nome_cidades_mg = [
    'Andrelândia', 'Araguari', 'Araxá', 'Araçuaí', 'Belo Horizonte',
    'Bambuí', 'Barbacena', 'Canápolis', 'Carmo do Paranaíba',
    'Cabeceira Grande', 'Cajuri', 'Cambuquira', 'Campo Belo',
    'Campos Altos', 'Caratinga', 'Conselheiro Lafaiete', 'Divinópolis',
    'Dores do Indaiá', 'Estrela do Sul', 'Formiga',
    'Governador Valadares', 'Guarda-Mor', 'Ituiutaba', 'Ipatinga',
    'Itabira', 'Itambacuri', 'Jacutinga', 'Januária', 'João Pinheiro',
    'Lavras', 'Luz', 'Monte Carmelo', 'Montes Claros', 'Mantena',
    'Mato Verde', 'Monte Alegre de Minas', 'Nova Resende',
    'Patos de Minas', 'Paineiras', 'Paracatu', 'Patrocínio',
    'Poços de Caldas', 'Raul Soares', 'Riachinho', 'Rio Casca',
    'Sacramento', 'Santa Fé de Minas', 'Santa Juliana', 'Sete Lagoas',
    'São Gonçalo do Sapucaí', 'São João del Rei', 'Taiobeiras',
    'Timóteo', 'Três Pontas', 'Tupaciguara', 'Uberaba', 'Uberlândia',
    'Unaí', 'Vazante'
]

nome_ascii_cidades_mg = get_normalized_names(nome_cidades_mg)

cidades_mg_final = generate_final_cidades_dataframe(nome_cidades_mg, nome_ascii_cidades_mg, 'MG')

In [3115]:
cidades_mg_final.sample(n=10)

,nome,nome_ascii,sigla_uf
55,Uberaba,uberaba,MG
23,Ipatinga,ipatinga,MG
13,Campos Altos,campos altos,MG
21,Guarda-Mor,guarda mor,MG
51,Taiobeiras,taiobeiras,MG
18,Estrela do Sul,estrela do sul,MG
50,São João del Rei,sao joao del rei,MG
34,Mato Verde,mato verde,MG
58,Vazante,vazante,MG
43,Riachinho,riachinho,MG


In [3116]:
nome_desnormalizado_map = list(zip(cidades_mg["nome"], cidades_mg["nome_normalizado"]))

cidades_mg_final['aliases'] = cidades_mg_final.apply(create_aliases, args=(85, nome_desnormalizado_map), axis=1)

In [3117]:
cidades_mg_final

,nome,nome_ascii,sigla_uf,aliases
0,Andrelândia,andrelandia,MG,[Andrelândia]
1,Araguari,araguari,MG,[Araguari]
2,Araxá,araxa,MG,[Araxá]
3,Araçuaí,aracuai,MG,[Araçuaí]
4,Belo Horizonte,belo horizonte,MG,"[BELO HORIZONTE, Belo Horizonte]"
5,Bambuí,bambui,MG,[Bambuí]
6,Barbacena,barbacena,MG,[Barbacena]
7,Canápolis,canapolis,MG,[CANÁPOLIS]
8,Carmo do Paranaíba,carmo do paranaiba,MG,"[CARMO DO PARANAIBA, Carmo do Paranaíba]"
9,Cabeceira Grande,cabeceira grande,MG,[Cabeceira Grande]


### Goiás - GO

In [3118]:
cidades_go = cidades[cidades['sigla_uf'] == 'GO']

In [3119]:
cidades_go['nome_normalizado'].unique()

array(['anapolis', 'aparecida de goiania', 'abadiania', 'alexania',
       'anicuns', 'bela vista de goias', 'buriti alegre', 'catalao',
       'caturai', 'ceres', 'cachoeira alta', 'caiaponia', 'caiponia',
       'calcilandia', 'caldas novas', 'campos belos',
       'carmo do rio verde', 'cacu', 'ceres go', 'cidade de goias',
       'cidade de goias goias velho', 'crixas', 'cumari',
       'corrego do ouro', 'damolandia', 'edeia', 'formosa',
       'firminopolis', 'formosa go', 'formoso', 'goiandira', 'goiania',
       'goianesia', 'goias', 'goianesiago', 'goiatuba', 'goiana',
       'goiania go', 'goania', 'hidrolandia', 'inhumas', 'itapuranga',
       'itumbiara', 'indianopolis', 'indiara', 'ipameri', 'ipora',
       'ipora go', 'itaberai', 'itaguaru', 'itapaci', 'itaucu', 'jaragua',
       'jatai', 'jandaia', 'jesupolis', 'joviania', 'jussara', 'luziania',
       'luziania go', 'morrinhos', 'mineiros', 'moipora', 'mossamedes',
       'mozarlandia', 'nova aurora', 'nazario', 'neropo

In [3120]:
cidades_go = cidades_go.drop(cidades_go[cidades_go['nome_normalizado'].isin(['indianopolis', 'porto nacional'])].index)

In [3121]:
nome_cidades_go = [
    'Anápolis', 'Aparecida de Goiânia', 'Abadiânia', 'Alexânia',
    'Anicuns', 'Bela Vista de Goiás', 'Buriti Alegre', 'Catalão',
    'Caturaí', 'Ceres', 'Cachoeira Alta', 'Caipônia',
    'Calcilândia', 'Caldas Novas', 'Campos Belos', 'Carmo do Rio Verde',
    'Caçu', 'Crixás', 'Cumari', 'Córrego do Ouro', 'Damolândia', 'Edéia',
    'Formosa', 'Firminópolis', 'Formoso', 'Goiandira',
    'Goiânia', 'Goiás', 'Goianésia', 'Goiatuba', 'Hidrolândia', 'Inhumas', 'Itapuranga',
    'Itumbiara', 'Indiara', 'Ipameri', 'Iporá',
    'Iporá', 'Itaberaí', 'Itaguari', 'Itapaci', 'Itauçu', 'Jaraguá',
    'Jataí', 'Jandaia', 'Jesúpolis', 'Joviânia', 'Jussara', 'Luziânia', 'Morrinhos',
    'Mineiros', 'Moiporá', 'Mossâmedes', 'Mozarlândia', 'Nova Aurora', 'Nazário', 'Nerópolis', 'Orizona',
    'Posse', 'Palmeiras de Goiás', 'Palminópolis',
    'Panamá', 'Paraúna', 'Petrolina de Goiás',
    'Pires do Rio', 'Piracanjuba', 'Pirenópolis', 'Pires do Rio',
    'Pontalina', 'Porangatu', 'Portelândia',
    'Quirinópolis', 'Rio Verde', 'Rialma', 'Rubiataba',
    'Santa Helena de Goiás', 'Santa Tereza de Goiás',
    'São Luís de Montes Belos', 'Silvânia', 'São Francisco de Goiás',
    'São Miguel do Araguaia', 'São Simão', 'Trindade', 'Uruaçu',
    'Urutaí', 'Vianópolis'
]

nome_ascii_cidades_go = get_normalized_names(nome_cidades_go)

cidades_go_final = generate_final_cidades_dataframe(nome_cidades_go, nome_ascii_cidades_go, 'GO')

In [3122]:
cidades_go_final.sample(n=30)

,nome,nome_ascii,sigla_uf
78,Silvânia,silvania,GO
64,Pires do Rio,pires do rio,GO
2,Abadiânia,abadiania,GO
53,Mozarlândia,mozarlandia,GO
21,Edéia,edeia,GO
55,Nazário,nazario,GO
41,Itauçu,itaucu,GO
81,São Simão,sao simao,GO
79,São Francisco de Goiás,sao francisco de goias,GO
75,Santa Helena de Goiás,santa helena de goias,GO


In [3152]:
cidades_go_starts_a = cidades_go[cidades_go['nome_normalizado'].str.startswith('e')]
cidades_go_final_starts_a = cidades_go_final[cidades_go_final['nome_ascii'].str.startswith('e')].iloc[:, :]

In [3144]:
def create_aliases_a(row, similarity_threshold, nome_desnormalizado_map, debug=False):
    nome_ascii = row["nome_ascii"]

    nao_encaixadas = []
    
    # aliases = [nome_desnormalizado for nome_desnormalizado, nome_normalizado in
    #            nome_desnormalizado_map if fuzz.ratio(nome_normalizado, nome_ascii) >= similarity_threshold]

    aliases = []
    
    for nome_desnormalizado, nome_normalizado in nome_desnormalizado_map:
        similarity_ratio = fuzz.ratio(nome_normalizado, nome_ascii)
        
        if similarity_ratio >= similarity_threshold:
            aliases.append(nome_desnormalizado)
        elif similarity_ratio in range (60, similarity_threshold):
            nao_encaixadas.append((nome_desnormalizado, similarity_ratio))

    if debug:
        print('-' * 5)
        print(f"cidade: {nome_ascii}")
        print(f"quantidade de elementos do mapa: {len(nome_desnormalizado_map)}")
        print(f"qtd aliases encontradas: {len(aliases)}")
        print(f"aliases encontradas: {aliases}")
        print(f"aliases não encaixadas: {nao_encaixadas}")
        print('-' * 5)
    
    return aliases

In [3153]:
nome_desnormalizado_map = list(zip(cidades_go_starts_a["nome"], cidades_go_starts_a["nome_normalizado"]))
# a = 90, b = 90, c = 75, d = 90,
cidades_go_final_starts_a['aliases'] = cidades_go_final_starts_a.apply(create_aliases_a, args=(90, nome_desnormalizado_map, True), axis=1)

-----
cidade: edeia
quantidade de elementos do mapa: 2
qtd aliases encontradas: 2
aliases encontradas: ['EDÉIA', 'Edéia']
aliases não encaixadas: []
-----


In [ ]:
# pegar uma lista com as iniciais que possuem municípios no dataset x
# particionar o dataframe original de cidades pelas iniciais x
# particionar o dataframe final de cidades pelas iniciais x
# particionar o mapa de nomes desnormalizados pelas iniciais x
# executar a inserção das labels em cada partição
# juntar tudo isso em um dataframe final

In [3157]:
cidades_go_iniciais = sorted(list(set(cidades_go['nome_normalizado'].str[0])))

In [3162]:
cidades_go_por_inicial = {inicial: pd.DataFrame(columns=cidades_go.columns, 
                                                data=cidades_go[cidades_go['nome_normalizado'].str.startswith(inicial)]) for inicial in cidades_go_iniciais}

In [3171]:
cidade_go_final_por_inicial = {inicial: pd.DataFrame(columns=cidades_go_final.columns,
                                                data=cidades_go_final[cidades_go_final['nome_ascii'].str.startswith(inicial)]) for inicial in cidades_go_iniciais}

In [3173]:
nome_desnormalizado_por_inicial_map = {inicial: list(zip(cidades_go_por_inicial[inicial]["nome"], cidades_go_por_inicial[inicial]["nome_normalizado"])) for inicial in cidades_go_iniciais}